### Prompt

Build a Bingo Card! This challenge uses events/observations from a baseball game, but you can customize this input for anything from traditional numbers to things you might see at a holiday party! 

Requirements:
1) Give the user a way to enter the number of cards to print (can be an app, workflow constant, text input, etc.)
2) Weight the list of items so each card contains 15 high-probability items, 5 medium-probability items, 3 low-probability items, and one super-rare item (24 total per card)
3) Within each probability group, randomly select the items
4) Add a free space in the middle (space #13) of the card
5) Build the cards with Alteryx reporting tools and render the results to a pdf, with one card per page

BEGINNER:
* Create 10 sets of randomly-selected items (does not have to be weighted)

INTERMEDIATE:
* Give the user a way to input the number of cards
* Apply the weighting logic in #2-3
* Include the free space in the middle
* Output the results to a basic table using reporting tools

ADVANCED:
* Meet all intermediate requirements
* Use reporting tools to build the card. Try to match the given output (or design your own!)

In [41]:
# importing lib

import pandas as pd

import os

import numpy as np

In [6]:
# importing data

cwd = os.getcwd()

file_path = os.path.join(cwd, 'data.csv')

df = pd.read_csv(file_path)

df

,Item,Type,Probability
0,B-Rush wins the tool race,Between-innings fun,Low
1,Phil the Bucket hits another tool,Between-innings fun,High
2,Fan wins the cap shuffle,Between-innings fun,Medium
3,Organist plays a song from a movie,Between-innings fun,Medium
4,Organist plays a song by Billy Joel,Between-innings fun,Medium
...,...,...,...
95,Outfielder throws someone out,Game,Low
96,Player wears glasses,Game,Medium
97,One of the teams bats around in a half inning,Game,Low
98,Wild pitch or passed ball,Game,Low


In [35]:
df_copy = df.copy()

In [7]:
df['Probability'].unique()

array(['Low', 'High', 'Medium', 'Super rare'], dtype=object)

In [95]:
# building function 

def bingo_builder(number_of_cards):
    
    dfs = []

    for i in range(1, number_of_cards + 1):

        high_prob = df[df['Probability'] == 'High'] # filtering high prob items
        high_prob = high_prob.sample(16, replace = False) # sampling

        med_prob = df[df['Probability'] == 'Medium'] # filtering medium prob items
        med_prob = med_prob.sample(5, replace = False) # sampling

        low_prob = df[df['Probability'] == 'Low'] # filtering low prob items
        low_prob = low_prob.sample(3, replace = False) # sampling

        super_rare = df[df['Probability'] == 'Super rare'] # filtering super rare items
        super_rare = super_rare.sample(1, replace = False) # sampling

        # concating dfs

        probs = [high_prob, med_prob, low_prob, super_rare]

        df_l = pd.concat(probs)

        # sorting and resetting index

        df_l.sort_values(by = 'Probability')

        df_l.reset_index(drop = True, inplace = True)

        # if index == 13, setting to free slot

        df_l.loc[df_l.index[13], 'Item'] = 'Free'

        # scrambling 

        df_l['order'] = df_l.apply(lambda x: 13 if x.name == 13 else np.random.choice([x for x in range(1, 25) if x != 13]), axis = 1)
       
        # setting player id
        
        df_l['player'] = i
        
        # appending df to list
        
        dfs.append(df_l)
        
    bingo_cards = pd.concat(dfs)
    
    return bingo_cards
    

In [96]:
# applying function

bingo_cards = bingo_builder(3)

In [97]:
bingo_cards.shape

(75, 5)

In [98]:
bingo_cards['player'].unique()

array([1, 2, 3], dtype=int64)

In [101]:
bingo_cards[bingo_cards['player'] == 3].sort_values(by = ['player', 'order'])

,Item,Type,Probability,order,player
2,The Freeze outruns fan,Game,High,1,3
1,Fan eating nachos,Crowd,High,2,3
10,Walk-up song is Latin,Game,High,5,3
20,Diving catch,Game,Medium,5,3
12,Fan eating hot dog,Crowd,High,6,3
6,Player's batting helmet comes off,Game,High,7,3
16,Home run,Game,Medium,8,3
18,Big screen shows food on grill,Game,Medium,8,3
22,Fielding error,Game,Low,8,3
23,Outfielder throws someone out,Game,Low,8,3
